https://facebook.github.io/prophet/docs/quick_start.html

In [ ]:
# Python
import mlflow.prophet
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_components_plotly
from sklearn.metrics import mean_squared_error
from src.app.app_utils import plot_plotly_with_testset

## 데이터 불러오기

In [ ]:
# Python
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df.head()

## 학습, 테스트셋 준비

In [ ]:
df['ds'] = pd.to_datetime(df.ds)
df_train = df[df.ds < "2015-01-01"]
df_test = df[(df.ds >= "2015-01-01") & (df.ds < "2016-01-01")]

## 모델 학습 및 평가

In [ ]:
# 파라미터 조정
params = {
    # "growth": "linear",
    # "changepoint_prior_scale": 0.5,
    # "seasonality_prior_scale": 0.1
}

In [ ]:
experiment = mlflow.get_experiment_by_name("train_prophet_model")
if experiment is None:
    experiment_id = mlflow.create_experiment("train_prophet_model")
    print("Experiment train_prophet_model is created")
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(experiment_id=experiment_id) as run:
    model = Prophet(**params)
    model.fit(df_train)

    future = model.make_future_dataframe(periods=365, include_history=False)
    forecast = model.predict(future)

    results = pd.merge(
        df_test[['ds','y']],
        forecast[['ds','yhat']],
        on='ds', how="left"
        )
    rmse_ = mean_squared_error(results.y, results.yhat)
    print(rmse_)
    
    mlflow.log_metric("rmse", rmse_)
    mlflow.prophet.log_model(model, artifact_path="prophet_model")

    # 모델 등록
    mlflow.register_model(
            f"runs:/{run.info.run_id}/prophet_model", "prophet_model"
        )

## 시각화

In [ ]:
future = model.make_future_dataframe(periods=365, include_history=True)
forecast = model.predict(future)
plot_plotly_with_testset(model, forecast, df_test)

In [ ]:
plot_components_plotly(model, forecast)